<a href="https://colab.research.google.com/github/KojoBarbie/CNN_practice_1/blob/main/CNN_practice_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNを用いて画像分類
pokeapiを使用して集めたポケモンの写真をもとに、そのポケモンが伝説のポケモンであるかどうかを判定する画像分類を作る。

## import
必要なモジュールなどのインポート

In [ ]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# 定番
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# CNNの時に使う
import torch
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data as data_utils#データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from PIL import Image

Mounted at /content/drive


## モデルを作るぞ
CNNでモデルを作って画像分類を行う。

In [ ]:
class Net(nn.Module):
    # NNの各構成要素を定義
    def __init__(self):
        super(Net, self).__init__()

        # 畳み込み層とプーリング層の要素定義
        self.conv1 = nn.Conv2d(3, 6, 5)  # (入力, 出力, 畳み込みカーネル（5*5）)
        self.pool = nn.MaxPool2d(2, 2)  # (2*2)のプーリングカーネル
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 全結合層の要素定義
        # self.fc1 = nn.Linear(61, 120)  # (入力, 出力)
        self.fc1 = nn.Linear(16 * 47 * 47, 120)  # (入力, 出力)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)  # クラス数が2なので最終出力数は2

    # この順番でNNを構成
    def forward(self, x):
        # print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))  # conv1->relu->pool
        # print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))  # conv2->relu->pool
        #print(x.shape)
        x = x.view(-1, 16 * 47 * 47)  # データサイズの変更
        # print(x.shape)
        x = F.relu(self.fc1(x))  # fc1->relu
        x = F.relu(self.fc2(x))  # fc2->relu
        x = self.fc3(x)
        return F.log_softmax(x, dim=0)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/10_study/CNN/data.csv")

train_data, test_data = train_test_split(df,train_size=0.25, test_size=0.05, stratify=df["legend"], random_state=30)
test_data.head()

,Unnamed: 0,id,img,legend
1266,1266,646,/content/drive/MyDrive/10_study/CNN/input/home...,1
13766,13766,287,/content/drive/MyDrive/10_study/CNN/input/artw...,0
19697,19697,97,/content/drive/MyDrive/10_study/CNN/input/home...,0
7237,7237,458,/content/drive/MyDrive/10_study/CNN/input/artw...,0
10963,10963,387,/content/drive/MyDrive/10_study/CNN/input/artw...,0


In [ ]:
#画像の変形処理
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
train_labels = torch.Tensor(train_data["legend"].values)
test_labels =  torch.Tensor(test_data["legend"].values)

In [ ]:
train_images = []
test_images = []

for i, file in enumerate(test_data["img"]):
        image = Image.open(file)
        image = transform(image)
        test_images.append(image)

In [ ]:
for i, file in enumerate(train_data["img"]):
        image = Image.open(file)
        image = transform(image)
        train_images.append(image)

In [ ]:
train_features = torch.stack(train_images)
train =  data_utils.TensorDataset(train_features, train_labels)
test_features = torch.stack(test_images)
test =  data_utils.TensorDataset(test_features, test_labels)

trainloader = data_utils.DataLoader(train, batch_size=32, shuffle=True, num_workers=2)
testloader = data_utils.DataLoader(test, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
#モデル定義
model = Net()
#GPU設定
device = 'cuda:0' 
model.to(device)
#Loss関数の指定
criterion = nn.CrossEntropyLoss()
#Optimizerの指定
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#トレーニング
#エポック数の指定
for epoch in range(100):  # loop over the dataset multiple times

    #データ全てのトータルロス
    running_loss = 0.0 


    for i, data in enumerate(trainloader):

        #入力データ・ラベルに分割
        # get the inputs
        inputs, labels = data

        # Variableに変形
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        # device設定
        inputs, labels = inputs.to(device), labels.to(device)

        # optimizerの初期化
        # zero the parameter gradients
        optimizer.zero_grad()

        #一連の流れ
        # forward + backward + optimize
        outputs = model(inputs)

        #ここでラベルデータに対するCross-Entropyがとられる
        loss = criterion(outputs, labels.type(torch.long))
        loss.backward()
        optimizer.step()

        # ロスの表示
        # print statistics
        running_loss += loss.data.item()
    if epoch % 25 == 0:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))


print('Finished Training')
print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / 2000))
print('Finished Training')

[1,   225] loss: 0.063
[26,   225] loss: 0.009
[51,   225] loss: 0.003
[76,   225] loss: 0.001
Finished Training
[100,   225] loss: 0.000
Finished Training


In [ ]:
# モデルを保存しておく
save_path = "/content/drive/MyDrive/10_study/CNN/output/predict_model.pth"
torch.save(model.state_dict(), save_path) # 保存

In [ ]:
# Accuracyの導出
correct = 0
total = 0
with torch.no_grad():
    for (images, labels) in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: {:.2f} %'.format(100 * float(correct/total)))

Accuracy: 90.32 %
